In [1]:
import polars as pl
import pyodbc
from datetime import date

### Definición de Rutas

In [5]:
cierre = "202504"
cierre_l = "Abr25"
cierre_d = date(2025,4,30)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/fotos/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperadas_detalle = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'
fl_pagadas_detalle = wd_data_processed_dwh + f'parquet/Pagadas_Global_VF_{cierre}.parquet'

# Outputs


### Importa tablas

In [4]:
schema_recuperadas = {
    'Concatenar_Saldos': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Programa_Id': pl.Int32,
    'Tipo_Garantia_Id': pl.Int16,
    'Tipo_Cambio_Cierre': pl.Float32,
    'Programa_Original': pl.Int32,
    'Porcentaje_Garantizado': pl.Float32,
    'Monto _Credito_Mn': pl.Float64,
    'Fecha_Apertura': pl.Datetime,
    'Moneda_Id': pl.Int16,
    'Tipo_Credito_Id': pl.Int16,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Int32,
    'NR_R': pl.String,
    'Producto': pl.String,
    'AGRUPAMIENTO_ID': pl.Float64,
    'AGRUPAMIENTO': pl.String,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Id': pl.Int16,
    'Monto': pl.Float64,
    'Interes': pl.Float64,
    'Moratorios': pl.Float64,
    'Descripcion': pl.String,
    'Estatus': pl.String,
    'Fecha_Registro': pl.Datetime,
    'Fecha': pl.Datetime,
    'Monto_Mn': pl.Float64,
    'Interes_Mn': pl.Float64,
    'Excedente': pl.Float64,
    'Excedente_Mn': pl.Float64,
    'Moratorios_Mn': pl.Float64,
    'Sub_Total_Mn': pl.Float64,
    'RazÃ³n Social (Intermediario)': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'TIPO_PERSONA': pl.String,
    'Recup_Rescat': pl.String,
    'MM_UDIS': pl.Int64,
    'ESQUEMA': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Gastos_Juicio_Mn': pl.Float64,
    'Monto_Total_Mn': pl.Float64,
    'MPagado_Mn': pl.Float64,
    'Historico': pl.String,
    'Fecha Registro Alta': pl.Datetime}

schema_pagadas = {
    'Concatenado': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Producto': pl.String,
    'Pago ID': pl.Int16,
    'RazÃ³n Social (Intermediario)': pl.String,
    'MIN_Fecha_Registro': pl.Datetime,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Float32,
    'Programa_Original': pl.Int32,
    'Programa_Id': pl.Int32,
    'Monto _Credito_Mn': pl.Float64,
    'Moneda_Id': pl.Int16,
    'Fecha de Apertura': pl.Datetime,
    'Tipo_Garantia_Id': pl.Int16,
    'TIPO_PERSONA': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'Monto_Desembolsado': pl.Float64,
    'Interes_Desembolso': pl.Float64,
    'Interes_Moratorios': pl.Float64,
    'Porcentaje_Garantizado': pl.Float32,
    'Tipo_Credito_Id': pl.Int16,
    'Estatus_Recuperacion': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'Fecha Registro Alta': pl.Datetime,
    'TC': pl.Float32,
    'AGRUPAMIENTO_ID': pl.Float32, # Revisar
    'AGRUPAMIENTO': pl.String,
    'ESQUEMA': pl.String,
    'SUBESQUEMA': pl.String,
    'CAMBIO': pl.Float64,
    'MM_UDIS': pl.Int16,
    'NR_R': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Monto_Desembolso_Mn': pl.Float64,
    'Interes_Desembolso_Mn': pl.Float64,
    'Interes_Moratorios_Mn': pl.Float64,
    'Monto_Pagado_Mn': pl.Float64
}

In [7]:
recuperadas_detalle = pl.read_parquet(fl_recuperadas_detalle)

In [8]:
recuperadas_detalle

Concatenar_Saldos,Fecha_Consulta,Programa_Id,Tipo_Garantia_Id,Tipo_Cambio_Cierre,Programa_Original,Porcentaje_Garantizado,Monto _Credito_Mn,Fecha_Apertura,Moneda_Id,Tipo_Credito_Id,Fecha_Garantia_Honrada,TPRO_CLAVE,NR_R,Producto,AGRUPAMIENTO_ID,AGRUPAMIENTO,Intermediario_Id,Numero_Credito,Id,Monto,Interes,Moratorios,Descripcion,Estatus,Fecha_Registro,Fecha,Monto_Mn,Interes_Mn,Excedente,Excedente_Mn,Moratorios_Mn,Sub_Total_Mn,Razón Social (Intermediario),Empresa / Acreditado (Descripción),RFC Empresa / Acreditado,TIPO_PERSONA,Recup_Rescat,MM_UDIS,ESQUEMA,CSG,CSF,Gastos_Juicio_Mn,Monto_Total_Mn,MPagado_Mn,Historico,Fecha Registro Alta
str,datetime[μs],i32,i32,f32,i32,f32,f64,datetime[μs],i64,i32,datetime[μs],i32,str,str,f64,str,str,str,i64,f64,f64,f64,str,str,datetime[μs],datetime[μs],f64,f64,f64,f64,f64,f64,str,str,str,str,str,i32,str,str,str,f64,f64,f64,str,datetime[μs]
"""1004020012709009602672806""",2025-04-30 00:00:00,3253,3,1.0,3999,100.0,20233.33,2008-03-19 00:00:00,1,25,2010-05-10 00:00:00,3253,"""NR""","""GARANTIA EMERGENTES""",1335.0,"""Emergente 2007 Tabasco""","""10040200""","""12709009602672806""",2,-5547.65,-69.34,null,"""CS P.253 - 1063733 DEP 2408201…","""CS""",2010-08-24 00:00:00,2010-08-24 00:00:00,-5547.65,-69.34,0.0,0.0,null,-5616.99,"""BANCO AZTECA""","""BONIFACIO HERNANDEZ DIONICIO""","""HEDB-721016-""","""F""","""Rescatada""",0,"""PP""","""CG""","""CF""",0.0,-5616.99,-5616.99,"""H""",2008-04-18 00:00:00
"""1004020012709009602663086""",2025-04-30 00:00:00,3253,3,1.0,3999,100.0,111283.33,2008-01-17 00:00:00,1,25,2010-02-12 00:00:00,3253,"""NR""","""GARANTIA EMERGENTES""",1335.0,"""Emergente 2007 Tabasco""","""10040200""","""12709009602663086""",2,-34220.15,-900.33,null,"""CS P.253 - 1063733 DEP 2408201…","""CS""",2010-08-24 00:00:00,2010-08-24 00:00:00,-34220.15,-900.33,0.0,0.0,null,-35120.48,"""BANCO AZTECA""","""JOSE ANTONIO GERONIMO TORRES""","""GETJ-480906-""","""F""","""Rescatada""",0,"""PP""","""CG""","""CF""",0.0,-35120.48,-34640.11,"""H""",2008-07-08 00:00:00
"""1004020012762089602666527""",2025-04-30 00:00:00,3253,3,1.0,3999,100.0,101166.67,2008-02-11 00:00:00,1,25,2009-03-12 00:00:00,3253,"""NR""","""GARANTIA EMERGENTES""",1335.0,"""Emergente 2007 Tabasco""","""10040200""","""12762089602666527""",1,6423.66,0.0,null,"""RP P.253 - 1063733 DEP 1408200…","""D""",2009-09-02 00:00:00,2009-09-02 00:00:00,6423.66,0.0,0.0,0.0,null,6423.66,"""BANCO AZTECA""","""FRANCISCO LUNA AVALOS""","""LUAF-671010-""","""F""","""Recuperada""",0,"""PP""","""CG""","""CF""",null,null,-75300.66,"""H""",2008-04-28 00:00:00
"""1004020012709009602671295""",2025-04-30 00:00:00,3253,3,1.0,3999,100.0,91050.0,2008-03-06 00:00:00,1,25,2010-05-10 00:00:00,3253,"""NR""","""GARANTIA EMERGENTES""",1335.0,"""Emergente 2007 Tabasco""","""10040200""","""12709009602671295""",1,338.71,0.0,null,"""Recuperacion automatica, carga…","""D""",2010-06-18 00:00:00,2010-06-02 00:00:00,338.71,0.0,0.0,0.0,null,338.71,"""BANCO AZTECA""","""ROBERTO MELENDREZ MONTES""","""MEMR-770620-""","""F""","""Recuperada""",0,"""PP""","""CG""","""CF""",0.0,338.71,-17784.22,"""H""",2008-04-18 00:00:00
"""1004020012709009602663292""",2025-04-30 00:00:00,3253,3,1.0,3999,100.0,151750.0,2008-01-21 00:00:00,1,25,2010-08-12 00:00:00,3253,"""NR""","""GARANTIA EMERGENTES""",1335.0,"""Emergente 2007 Tabasco""","""10040200""","""12709009602663292""",1,170.89,0.0,null,"""Recuperacion automatica, carga…","""D""",2010-09-07 00:00:00,2010-09-06 00:00:00,170.89,0.0,0.0,0.0,null,170.89,"""BANCO AZTECA""","""ESTHER CAROLINA JASSO GARCIA""","""JAGE-661123-""","""F""","""Recuperada""",0,"""PP""","""CG""","""CF""",0.0,170.89,-1908.45,"""H""",2008-06-13 00:00:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""604239589666359810""",2025-04-30 00:00:00,390160,4,1.0,3999,60.0,2.5e6,2024-05-31 00:00:00,1,2,2024-11-13 00:00:00,390160,"""NR""","""GARANTIAS BANCOMEXT""",1225.0,"""Bancomext""","""60423958""","""9666359810""",2,18608.82,0.0,0.0,"""Re

In [5]:
df_recuperadas_detalle = pl.read_csv(fl_recuperadas_detalle, schema_overrides=schema_recuperadas)
df_recuperadas_detalle = (df_recuperadas_detalle
                          .filter(
                            (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                            (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                            (pl.col("Producto") != "GARANTIAS BANSEFI")
                          ))

In [6]:
df_pagadas_detalle = pl.read_csv(fl_pagadas_detalle, schema_overrides=schema_pagadas)
df_pagadas_detalle = (df_pagadas_detalle
                          .filter(
                            (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                            (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                            (pl.col("Producto") != "GARANTIAS BANSEFI")
                          ))

In [ ]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)
llave = pl.read_database(query='SELECT * FROM [LLAVE]', connection=conn)
conn.close()



In [4]:
fl_recuperadas_detalle

'E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/parquet/Recupera_con_Pagos_Flujos_202504.csv'